# Project Overview


## 1. Import Python Packages

In [3]:
from tensorflow import keras 
import keras_preprocessing
from keras_preprocessing import image
import tensorflow as tf
import natsort
from natsort import natsorted
from glob import glob
from skimage import img_as_float32
import imageio
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
# get training images and corresponding masks path
TRAIN_DATA_PATH = list(zip(
    natsorted(glob(f'./dataset/train/*.png')),
    natsorted(glob(f'./dataset/train_labels/*.png')),
))

VAL_DATA_PATH = list(zip(
    natsorted(glob(f'./dataset/val/*.png')),
    natsorted(glob(f'./dataset/val_labels/*.png')),
))

TEST_DATA_PATH= list(zip(
    natsorted(glob(f'./dataset/test/*.png')),
    natsorted(glob(f'./dataset/test_labels/*.png')),
))

## 2. Image Preprocessing

In [29]:
class ImageProcessing(object):
    """
    This is a blueprint for image processing
    """
    
    def __init__(self, IMAGES = None, MASKS = None):
        self.SIZE = (512,512)
        self.IMAGES = []
        self.MASKS = []
        
    """
    1.IMAGE PREPROCESSING
    """
    def get_images(self, PATH):
        """
        This method loads and converts images as a color image
        :returns: images as numpy array
        """
        
        # read all images and store them in an array. 
        # borrowed from https://realpython.com/storing-images-in-python/
        for img_path, _ in tqdm(PATH, "Loading images"):
            
            # load images as float32 and store it in an array
            image = img_as_float32(cv.imread(img_path, cv.IMREAD_COLOR))
            
            # downsample the input image to 512 x 512 x 3
            image = cv.resize(image, self.SIZE)
            
            # convert images to color images since opencv reads images grayscale
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
            
            # store all images
            self.IMAGES.append(image)
            
        # convert image to a numpy array
        IMGS = np.array(self.IMAGES)
        
        # check if all images have the same size and color images (RGB with 3 channels)
        assert IMGS.shape == (len(self.IMAGES), self.SIZE[0], self.SIZE[1], 3)
                
        
        print('\n', IMGS[0].shape)
        
        return IMGS
            
            
    def get_masks(self, PATH):
        """
        This method loads and converts images as a grayscale image
        :returns: masks as a numpy array      
        """
        
        # read all images and store them in an array. 
        # borrowed from https://realpython.com/storing-images-in-python/
        for _, msk_path  in tqdm(PATH, "Loading masks"):
            
            # load images as float32 and store it in an array
            mask = img_as_float32(cv.imread(msk_path, cv.IMREAD_COLOR))
            
            # downsample the input image to 512 x 512 
            mask= cv.resize(mask, self.SIZE)
            
            # convert masks to grayscale 
            mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
            
            # store all masks
            self.MASKS.append(mask)
            
        # convert masks to a numpy array
        MSKS = np.array(self.MASKS)
        
        # check if all masks have the same size
        #assert MSKS.shape == (len(self.MASKS), self.SIZE[0], self.SIZE[1])
        assert MSKS.shape == (len(self.MASKS), self.SIZE[0], self.SIZE[1])
        
        # expand the dimensions of the masks
        MSKS = np.expand_dims(MSKS, axis=3)
        
        return MSKS
        
buildings_segmentation = BuildingsSegmentation()

In [6]:
images = buildings_segmentation.get_images(TRAIN_DATA_PATH)

Loading images: 100%|██████████| 131/131 [00:08<00:00, 15.46it/s]



 (512, 512, 3)


In [30]:
masks = buildings_segmentation.get_masks(TRAIN_DATA_PATH)

Loading masks: 100%|██████████| 131/131 [00:03<00:00, 35.02it/s]


In [ ]:

def visualize_images(images, masks):

    plt.figure(figsize=(20,16))
    x, y = 6,4

    for i in range(y):  
        for j in range(x):
            # images 
            plt.subplot(y*2, x, i*2*x+j+1)
            pos =  i+j
            plt.imshow(images[pos])
            plt.title('Image #{}'.format(pos))
            plt.axis('off')
            plt.subplot(y*2, x, (i*2+1)*x+j+1)
            
            # display corresponding masks
            plt.imshow(np.squeeze(masks[pos]), cmap="gray")
            plt.title('Mask #{}'.format(pos))
            plt.axis('off')
    plt.show()

visualize_images(resized_images, resized_masks)